In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # this is used for the plot the graph 
import seaborn as sns # used for plot interactive graph. 
from pandas import set_option
plt.style.use('ggplot') # nice plots
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', None)

dataset = pd.read_csv('/Users/nic/Documents/学校论文/代码/datasets/UCI_Credit_Card.csv')
dataset.head(5)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [171]:
X = dataset.drop(['default.payment.next.month'],axis=1)
y = dataset['default.payment.next.month']

import tensorflow as tf
%matplotlib inline
from keras.layers import Input, Dense, LSTM
from keras.models import Model

from keras.layers import *
from keras.models import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import optimizers
from sklearn.metrics import  accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
print('import done!')

import done!


In [172]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

scaler = MinMaxScaler(feature_range=(-1, 1))

output_dim = 1
batch_size = 256 #每轮训练模型时，样本的数量
epochs = 60 #训练60轮次
seq_len = 6 # 每3个为一组时间窗口
hidden_size = 128

TIME_STEPS = 6
INPUT_DIM = 2

lstm_units = 64
dataset_flow = dataset[['PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6',
#                         'BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6',
                        'PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6'
                       ]]
scaler.fit(dataset_flow)
X_train, X_test, y_train, y_test = train_test_split(dataset_flow, y, test_size = 0.2, random_state = 1)
# 归一化
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
train_num = X_train.shape[0]
test_num = X_test.shape[0]
X_train = X_train.reshape((train_num, seq_len, INPUT_DIM))
y_train = y_train.values
X_test = X_test.reshape((test_num, seq_len, INPUT_DIM))
y_test = y_test.values
# X_train = np.array([data_train[i : i + seq_len, :] for i in range(data_train.shape[0] - seq_len)])
# y_train = np.array([data_train[i + seq_len, 0] for i in range(data_train.shape[0]- seq_len)])
# X_test = np.array([data_test[i : i + seq_len, :] for i in range(data_test.shape[0]- seq_len)])
# y_test = np.array([data_test[i + seq_len, 0] for i in range(data_test.shape[0] - seq_len)])

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(24000, 6, 2) (24000,) (6000, 6, 2) (6000,)


In [173]:
from tensorflow.keras.layers import Dense, Lambda, Dot, Activation, Concatenate
from tensorflow.keras.layers import Layer


class Attention_(Layer):

    def __init__(self, units=128, **kwargs):
        self.units = units
        super().__init__(**kwargs)

    def __call__(self, inputs):
        """
        Many-to-one attention mechanism for Keras.
        @param inputs: 3D tensor with shape (batch_size, time_steps, input_dim).
        @return: 2D tensor with shape (batch_size, 128)
        @author: felixhao28, philipperemy.
        """
        hidden_states = inputs
        hidden_size = int(hidden_states.shape[2])
        # Inside dense layer
        #              hidden_states            dot               W            =>           score_first_part
        # (batch_size, time_steps, hidden_size) dot (hidden_size, hidden_size) => (batch_size, time_steps, hidden_size)
        # W is the trainable weight matrix of attention Luong's multiplicative style score
        score_first_part = Dense(hidden_size, use_bias=False, name='attention_score_vec')(hidden_states)
        #            score_first_part           dot        last_hidden_state     => attention_weights
        # (batch_size, time_steps, hidden_size) dot   (batch_size, hidden_size)  => (batch_size, time_steps)
        h_t = Lambda(lambda x: x[:, -1, :], output_shape=(hidden_size,), name='last_hidden_state')(hidden_states)
        score = Dot(axes=[1, 2], name='attention_score')([h_t, score_first_part])
        attention_weights = Activation('softmax', name='attention_weight')(score)
        # (batch_size, time_steps, hidden_size) dot (batch_size, time_steps) => (batch_size, hidden_size)
        context_vector = Dot(axes=[1, 1], name='context_vector')([hidden_states, attention_weights])
        pre_activation = Concatenate(name='attention_output')([context_vector, h_t])
        attention_vector = Dense(self.units, use_bias=False, activation='tanh', name='attention_vector')(pre_activation)
        return attention_vector

    def get_config(self):
        return {'units': self.units}

    @classmethod
    def from_config(cls, config):
        return cls(**config)
    

In [178]:
model_input = Input(shape=(TIME_STEPS, INPUT_DIM))
# model_input = Input(shape=[X_train.shape[1], X_train.shape[2],])
x = LSTM(32, return_sequences=True)(model_input)
# print(x.shape, type(x))
x = Attention_(32)(x)
x = Dense(1)(x)
model = Model(model_input, x)
print(model.summary())

Model: "model_17"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(None, 6, 2)]       0                                            
__________________________________________________________________________________________________
lstm_13 (LSTM)                  (None, 6, 32)        4480        input_18[0][0]                   
__________________________________________________________________________________________________
last_hidden_state (Lambda)      (None, 32)           0           lstm_13[0][0]                    
__________________________________________________________________________________________________
attention_score_vec (Dense)     (None, 6, 32)        1024        lstm_13[0][0]                    
___________________________________________________________________________________________

In [186]:
sgd = optimizers.SGD(lr=0.02, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer='adam')
cw = {0:1, 1:2}
model.fit(X_train, y_train, epochs=200, batch_size=256, shuffle=False, class_weight=cw)
y_pred = model.predict(X_test)
print('Train loss:', model.evaluate(X_train, y_train, batch_size=batch_size))
print('Test loss:', model.evaluate(X_test, y_test, batch_size=batch_size))
print('ROC-AUC:', roc_auc_score(y_test, y_pred))

/Users/nic/Library/Python/3.7/lib/python/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/200
94/94 [==============================] - 2s 4ms/step - loss: 0.6565
Epoch 2/200
94/94 [==============================] - 0s 4ms/step - loss: 0.6560
Epoch 3/200
94/94 [==============================] - 0s 4ms/step - loss: 0.6559
Epoch 4/200
94/94 [==============================] - 0s 4ms/step - loss: 0.6558
Epoch 5/200
94/94 [==============================] - 0s 4ms/step - loss: 0.6557
Epoch 6/200
94/94 [==============================] - 0s 4ms/step - loss: 0.6557
Epoch 7/200
94/94 [==============================] - 0s 4ms/step - loss: 0.6556
Epoch 8/200
94/94 [==============================] - 0s 4ms/step - loss: 0.6556
Epoch 9/200
94/94 [==============================] - 0s 4ms/step - loss: 0.6555
Epoch 10/200
94/94 [==============================] - 0s 4ms/step - loss: 0.6554
Epoch 11/200
94/94 [==============================] - 0s 4ms/step - loss: 0.6554
Epoch 12/200
94/94 [==============================] - 0s 4ms/step - loss: 0.6553
Epoch 13/200
94/94 [=================

94/94 [==============================] - 0s 4ms/step - loss: 0.6524
Epoch 103/200
94/94 [==============================] - 0s 4ms/step - loss: 0.6523
Epoch 104/200
94/94 [==============================] - 0s 4ms/step - loss: 0.6523
Epoch 105/200
94/94 [==============================] - 0s 4ms/step - loss: 0.6522
Epoch 106/200
94/94 [==============================] - 0s 4ms/step - loss: 0.6522
Epoch 107/200
94/94 [==============================] - 0s 4ms/step - loss: 0.6522
Epoch 108/200
94/94 [==============================] - 0s 4ms/step - loss: 0.6521
Epoch 109/200
94/94 [==============================] - 0s 4ms/step - loss: 0.6521
Epoch 110/200
94/94 [==============================] - 0s 4ms/step - loss: 0.6520
Epoch 111/200
94/94 [==============================] - 0s 4ms/step - loss: 0.6520
Epoch 112/200
94/94 [==============================] - 0s 4ms/step - loss: 0.6520
Epoch 113/200
94/94 [==============================] - 0s 4ms/step - loss: 0.6519
Epoch 114/200
94/94 [=========

24/24 [==============================] - 0s 2ms/step - loss: 0.4910
Test loss: 0.4909914433956146
ROC-AUC: 0.7550705108453362


In [189]:
from sklearn.metrics import classification_report
y_pred_flatten = y_pred.flatten()
print(y_test, '\n',y_pred)
print(len([y_p for y_p in y_pred if y_p!=0]))
y_pred_new = [0 if y_p<0.5 else 1 for y_p in y_pred_flatten]
print(classification_report(y_test, y_pred_new))

[0 1 0 ... 0 0 0] 
 [[0.19445398]
 [0.5939353 ]
 [0.26893526]
 ...
 [0.20419982]
 [0.24588855]
 [0.28427085]]
6000
              precision    recall  f1-score   support

           0       0.86      0.88      0.87      4663
           1       0.55      0.49      0.52      1337

    accuracy                           0.80      6000
   macro avg       0.70      0.69      0.69      6000
weighted avg       0.79      0.80      0.79      6000



In [190]:
from sklearn.metrics import  accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
roc=roc_auc_score(y_test, y_pred)
acc = accuracy_score(y_test, y_pred_new)
prec = precision_score(y_test, y_pred_new)
rec = recall_score(y_test, y_pred_new)
f1 = f1_score(y_test, y_pred_new)
print(roc,acc,prec,rec,f1)

0.7550705108453362 0.7955 0.5459098497495827 0.48915482423335827 0.5159763313609468
